In [1]:
import cv2
import pytesseract
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
import joblib
import tensorflow.keras as keras

# Step 1: Load the Text Model
text_model_file = 'text_model.pkl'
text_model = joblib.load(text_model_file)

# Step 2: Load the Nature Model
nature_model_file = 'nature_model1.h5'
nature_model = keras.models.load_model(nature_model_file)

# Step 3: Function to extract text from an image
def extract_text_from_image(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    text = pytesseract.image_to_string(gray)
    return text

# Step 4: Function to preprocess the text for prediction
def preprocess_text(text, vectorizer):
    text_features = vectorizer.transform([text])
    return text_features

# Step 5: Function to preprocess the image for nature classification
def preprocess_image(img):
    img = cv2.resize(img, (224, 224))
    img = img.astype('float32') / 255.0
    return np.expand_dims(img, axis=0)

# Step 6: Function to predict the class labels for text and nature
def predict_text_and_nature_classes(img, vectorizer):
    text = extract_text_from_image(img)
    text_features = preprocess_text(text, vectorizer)
    text_class_label = text_model.predict(text_features)[0]

    img = preprocess_image(img)
    nature_class_probabilities = nature_model.predict(img)
    nature_class_label = np.argmax(nature_class_probabilities, axis=-1)[0]

    return text_class_label, nature_class_label

# Load the TfidfVectorizer used during training
vectorizer_file = 'vectorizer.pkl'
vectorizer = joblib.load(vectorizer_file)

# Open the camera (use 0 for the default camera or provide the specific camera index)
camera = cv2.VideoCapture(0)

while True:
    # Capture a frame from the camera
    ret, frame = camera.read()
    
    # Check if the frame is captured successfully
    if not ret:
        break

    # Predict the class labels for text and nature
    text_class_label, nature_class_label = predict_text_and_nature_classes(frame, vectorizer)

   # Class names for text and nature classes
    text_class_names = ['Islamophobic image', 'islamophobic text', 'Non Islamophobic image', 'Non islamophobic text']
    nature_class_names = ['Islamophobic image', 'islamophobic text', 'Non islamophobic text', 'Non Islamophobic image'] 

    # Function to get the class names for text and nature classes
    def get_class_name(class_label, class_names):
        return class_names[class_label]

    # Get the class names for the predicted class labels
    text_class_name = get_class_name(text_class_label, text_class_names)
    nature_class_name = get_class_name(nature_class_label, nature_class_names)

    # Display the predicted class names on the frame
    cv2.putText(frame, f"Text: {text_class_name}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
    cv2.putText(frame, f"Nature: {nature_class_name}", (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

    # Show the frame
    cv2.imshow('Real-Time Classification', frame)

    # Break the loop if 'q' key is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the camera and close the window
camera.release()
cv2.destroyAllWindows()


1/1 [==============================] - 0s 46ms/step
